In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/Waterloo/_MSCI_641/project'

/content/drive/MyDrive/Waterloo/_MSCI_641/project


In [3]:
!pip install virtualenv
!pip install rouge_score
!pip install transformers
!pip install wandb
!pip install datasets
!pip install evaluate
!pip install accelerate
# !pip install gradio --upgrade
# !pip install --upgrade torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=297f5200884d27d274d15256b3ac816a976797fbaa03f86b7bf03fbad25679e1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [4]:
import numpy as np
import pandas as pd
from utils import *
import json, os, gc

import evaluate
import torch, wandb
from tqdm.auto import tqdm
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import default_data_collator
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForSequenceClassification, AdamW, RobertaTokenizer
from collections import defaultdict
from transformers import Trainer, TrainingArguments
from accelerate import Accelerator
from transformers import get_scheduler
# import gradio as gr
from sklearn.metrics import f1_score, accuracy_score

os.environ["TOKENIZERS_PARALLELISM"] = "false"

tqdm.pandas()

In [5]:
datasets = dict()
dataset_names = ['train','val','test']

for dataset_name in dataset_names:
    datasets[dataset_name] = pd.read_json(path_or_buf=dataset_name + '.jsonl', lines=True)
    if 'id' not in datasets[dataset_name].columns:
        datasets[dataset_name]['id'] = datasets[dataset_name].index

features = ['postText', 'postPlatform', 'targetParagraphs', 'targetTitle',
       'targetDescription', 'targetKeywords', 'targetMedia', 'targetUrl']
features = ['postText', 'postPlatform', 'targetParagraphs', 'targetTitle',
       'targetDescription', 'targetKeywords', 'targetMedia', 'targetUrl']
features_text = ['postText','targetParagraphs', 'targetTitle', 'targetDescription',
                'targetKeywords']
features_url = ['targetUrl', 'targetMedia' ]

target1 = 'tags'
target2 = 'spoiler'

In [8]:
label_encoder = {"phrase": 0,"passage": 1,"multi":2}
label_decoder = {0:"phrase",1:"passage", 2:"multi"}

for dataset_name in dataset_names:

    print(dataset_name)

    temp_data = datasets[dataset_name]

    if dataset_name != 'test':
        temp_data = temp_data[temp_data['spoilerPositions'].apply(lambda x: len(x[0])) > 1]
        temp_data['tokPos'] = temp_data.apply(findPosTags,1)

        temp_data['tags'] = temp_data['tags'].apply(lambda x: x[0])
        temp_data['label'] = temp_data['tags'].map(label_encoder)
        temp_data['tokenPosition'] = temp_data.apply(findPosTags,1)
        temp_data['longest_spoiler_index'] = temp_data['spoiler'].apply(return_longest_spoiler_index)
        temp_data['longest_spoiler'] = temp_data.apply(lambda x: x['spoiler'][x['longest_spoiler_index']],1)
        temp_data['longest_tokPos'] = temp_data.apply(lambda x: x['tokPos'][x['longest_spoiler_index']],1)

    temp_data['postText'] = temp_data['postText'].apply(lambda x: x[0])

    for col in ['targetParagraphs', 'targetMedia']:

        print(col)

        temp_data[col + '_listlen'] = temp_data[col].apply(lambda x: len(x) if x is not None else 0)
        temp_data[col + '_concat'] = temp_data[col].apply(lambda x: ' '.join(x) if x is not None else '')

    for col in ['postText', 'targetTitle', 'targetDescription', 'targetKeywords', 'targetParagraphs_concat']:

        print(col)
        temp_data[col + '_len'] = temp_data[col].apply(lambda x: len(x.split()) if x is not None else 0)

    temp_data['title_para'] = temp_data['targetTitle'] + " - " + temp_data['targetParagraphs_concat']

    datasets[dataset_name] = temp_data

train


<ipython-input-8-0d1bb7c1b72f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['tokPos'] = temp_data.apply(findPosTags,1)
<ipython-input-8-0d1bb7c1b72f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['tags'] = temp_data['tags'].apply(lambda x: x[0])
<ipython-input-8-0d1bb7c1b72f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

targetParagraphs
targetMedia
postText
targetTitle
targetDescription
targetKeywords
targetParagraphs_concat


<ipython-input-8-0d1bb7c1b72f>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data[col + '_len'] = temp_data[col].apply(lambda x: len(x.split()) if x is not None else 0)
<ipython-input-8-0d1bb7c1b72f>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['title_para'] = temp_data['targetTitle'] + " - " + temp_data['targetParagraphs_concat']


val
targetParagraphs
targetMedia
postText
targetTitle
targetDescription
targetKeywords
targetParagraphs_concat
test
targetParagraphs
targetMedia
postText
targetTitle
targetDescription
targetKeywords
targetParagraphs_concat


In [9]:
def convert2squadFormat_old(df):
    df_fin = df[['id','targetTitle','postText',"title_para","tokPos","spoiler"]]
    df_fin["asnwers"] = df_fin.apply(lambda x: {'text':x['spoiler'], "answer_start":[x['tokPos'][0][0]]},1)
    df_fin['id'] = df_fin['id'].astype(str)
    df_fin = df_fin.drop(columns=["tokPos","spoiler"])
    df_fin.columns = ['id',"title","question","context","answers"]
    return df_fin

In [10]:
datasets['test'].iloc[8]

,8
postId,53e76y
postText,The First Person In The Royal Family Has Just Come Out As Gay
postPlatform,reddit
targetParagraphs,"[The Queen’s cousin has become the first member of the British Royal Family to come out as gay., Lord Ivar Mountbatten has revealed that he is now dating a man after a lifelong struggle with his sexuality., Prince Harry previously joked that he might try a relationship with a man if it doesn’t work out with women., Mountbatten is the Queen’s cousin, the great great great grandson of Queen Victoria and the great-nephew of Earl Mountbatten of Burma., The 53-year-old came out in an interview with his boyfriend James Coyle, 54. The couple met on a skiing holiday in March, and the relationship has been a key landmark for them both., He is the first member of the extended royal family to come out as gay, and whilst he is happier now he’s still finding his feet in his new relationship., ""I am a lot happier now, though I am still not 100% comfortable with being gay,"" he told The Mail on Sunday., ""Being a Mountbatten was never the problem, it was the generation into which I was born. When I was growing up, it was known as ‘the love that dare not speak its name’, but what’s amazing now is how far we have all come in terms of acceptance."", ""Coming out is such a funny phrase but it’s what I suppose I did in a rather roundabout way, emerging to a place I’m happy to be. I have struggled with my sexuality and in some ways I still do; it has been a real journey to reach this point."", ""Simply talking about it in public is a huge step for me. Up to this point, I have had a heterosexual lifestyle, so living with a man is really new. One step at a time."", The father of three divorced from his wife Penelope Thompson after 17 years of marriage in 2011., Whilst having a boyfriend has provided the Lord with some learning curves, it’s also been a blessing in disguise. His daughters like having another man in the house, rather than a step-mother. And his ex, Penelope Thomson, goes to yoga with his new boyfriend., Coyle said: ""I went to a village yoga class with Penny and people thought I was her new boyfriend. She had to tell them: no, he’s Ivar’s, not mine."", Whilst the couple have a strict no public affection rule, they are now out to all of their friends and family., ""In different ways, we both struggled. Now everyone in our family knows and could not be more supportive."" said Mountbatten. ""Neither of us wanted to have relationships which were transitory; the stereotypical view of gay relationships is that they are too fleeting, too frequent."", ""Finding James means I will not have to lie to anyone or grow old on my own."", Last year, Prince William visited a school to participate in a lesson to tackle homophophic bullying., The Queen is believed to have supported same-sex marriage. When a newspaper claimed that she opposed gay people marrying, Buckingham Palace sent journalists PinkNews articles claiming that she supports equality.]"
targetTitle,The first person in the royal family has just come out as gay
targetDescription,The Queen’s cousin has become the first member of the royal family to come out as gay.
targetKeywords,None
targetMedia,"[http://cdn.pinknews.co.uk/images/2016/08/truvada-93x70.jpg, http://cdn.pinknews.co.uk/images/2015/07/close.png, http://cdn.pinknews.co.uk/wp-content/themes/pn-responsive-full/images/pn_mobile_logo.png, http://0.gravatar.com/avatar/fc7bd9632bf87b2e3280355bdac262e4?s=25&d=mm&r=g, http://cdn.pinknews.co.uk/images/2016/09/queens-cousin-1_640x345_acf_cropped.jpg, http://cdn.pinknews.co.uk/images/2016/07/harrydi-93x70.jpg, http://cdn.pinknews.co.uk/wp-content/themes/pn-responsive-full/images/pn_new_logo.png, http://pixel.quantserve.com/pixel/p-b9cuJqjzzENZA.gif, http://cdn.pinknews.co.uk/images/2016/07/Prince-Harry-HIV2-93x70.jpg, http://cdn.pinknews.co.uk/images/2016/06/150903-queen-elizabeth-jpo-712a_80834894bceed6fc1635f6a6ac96ab46.nbcnews-ux-2880-1000.jpg, http://cdn.pinknew

In [11]:
datasets['train']['tags']

,tags
0,passage
1,phrase
2,phrase
3,multi
4,phrase
5,passage
6,passage
7,passage
8,passage
9,multi


In [13]:
train_df = fixFormat(datasets['train'], 'train')
val_df = fixFormat(datasets['val'], 'val')
test_df = fixFormat(datasets['test'], 'test')

train_data = Dataset.from_pandas(train_df.reset_index(drop=True), split="train")
val_data = Dataset.from_pandas(val_df.reset_index(drop=True), split="val")
test_data = Dataset.from_pandas(test_df.reset_index(drop=True), split="test")

<ipython-input-12-341f58f099c6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin['id'] = df_fin['id'].astype(str)
<ipython-input-12-341f58f099c6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin['id'] = df_fin['id'].astype(str)
<ipython-input-12-341f58f099c6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [14]:
train_df

,id,label,title_para
0,0,1,"Wes Welker Wanted Dinner With Tom Brady, But P..."
1,1,0,Hole In Ozone Layer Expected To Make Full Reco...
2,2,0,Intellectual Stimulation Trumps Money For Empl...
3,3,2,"‘Follow your passion’ is wrong, here are 7 hab..."
4,4,0,Revealed: The perfect way to cook rice so that...
...,...,...,...
3195,3195,1,"Facebook Video Surging, But YouTube Still Offe..."
3196,3196,1,Cop is eating at Chili's when teen hands him f...
3197,3197,2,5 popular myths about visible signs of aging t...
3198,3198,0,"WTF, It Looks Like This Twitter Account ""Predi..."


In [17]:
model_name = "google-bert/bert-base-uncased"

# Para Generation
# config = dict(
# max_length = 512,
# stride = 128,
# n_best = 15,
# max_answer_length = 100,
# batch_size = 8,
# epochs = 20,
# learning_rate = 1e-6,
# model_name = model_name,
# spoiler_type = "passage"
# )

# Phrase Generation
config = dict(
max_length = 512,
stride = 128,
n_best = 10,
max_answer_length = 30,
batch_size = 16,
epochs = 10,
learning_rate = 1e-6,
model_name = model_name,
spoiler_type = "phrase"
)


max_length = config["max_length"]
stride = config["stride"]
n_best = config["n_best"]
max_answer_length = config["max_answer_length"]


tokenizer = AutoTokenizer.from_pretrained(config["model_name"])
model = AutoModelForSequenceClassification.from_pretrained(config["model_name"], num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [21]:
train_dataset = train_data.map(
    preprocess_training_examples_cls,
    batched=True,
    remove_columns=train_data.column_names)

Map:   0%|          | 0/3199 [00:00<?, ? examples/s]

In [22]:
validation_dataset = val_data.map(
    preprocess_training_examples_cls,
    batched=True,
    remove_columns=val_data.column_names,
)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [23]:
test_dataset = test_data.map(
    preprocess_test_examples_cls,
    batched=True,
    remove_columns=test_data.column_names,
)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

len(inputs["input_ids"]) 831


In [24]:
train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id', 'labels'],
    num_rows: 6449
})

In [25]:
test_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 831
})

In [26]:
validation_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id', 'labels'],
    num_rows: 838
})

In [27]:
len(train_dataset), len(validation_dataset), len(test_dataset)

(6449, 838, 831)

In [30]:
train_set = train_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
test_set = test_dataset.remove_columns(["example_id", "offset_mapping"])
train_set.set_format("torch")
validation_set.set_format("torch")
test_set.set_format("torch")

train_dataloader = DataLoader(
    train_set,
    collate_fn=default_data_collator,
    batch_size=config["batch_size"],
)

eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=32
)
test_dataloader = DataLoader(
    test_set, collate_fn=default_data_collator, batch_size=32
)

optimizer = AdamW(model.parameters(), lr=config["learning_rate"])
accelerator = Accelerator(mixed_precision ='fp16')
model, optimizer, train_dataloader, eval_dataloader, test_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, test_dataloader
)

num_train_epochs = config["epochs"]
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [33]:
outputs

SequenceClassifierOutput(loss=tensor(1.2686, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 0.8438,  0.0204, -0.4905],
        [ 0.9307, -0.1183, -0.4912],
        [ 1.0430, -0.2583, -0.5532],
        [ 0.7266, -0.0459, -0.5913],
        [ 0.6172,  0.1860, -0.2659],
        [ 0.8018, -0.0349, -0.3713],
        [ 0.2817,  0.0219, -0.6084],
        [ 0.5210, -0.0576, -0.3669],
        [ 0.6157, -0.1024, -0.2825],
        [-0.4395,  0.4243,  0.2510],
        [ 0.8604, -0.1907, -0.8374],
        [ 0.7188, -0.1203, -0.6729],
        [ 1.1436, -0.2827, -0.5103],
        [ 1.2422, -0.0415, -0.4822],
        [ 0.8330, -0.2036, -0.5908],
        [ 0.8516,  0.2089, -0.4597]], device='cuda:0',
       grad_fn=<ToCopyBackward0>), hidden_states=None, attentions=None)

In [46]:
stats = pd.DataFrame()

for epoch in range(num_train_epochs):
    print("Starting epoch: ", epoch)
    # Training
    model.train()
    train_loss = 0
    train_cls_loss = 0
    train_qa_loss = 0
    start_logits_t = []
    end_logits_t = []
    class_logits_t = []
    for batch in tqdm(train_dataloader):
        outputs = model(**batch)
        loss = outputs['loss']
        accelerator.backward(loss)
#        optimizer.zero_grad()
#        loss.backward()
#        optimizer.step()
        train_loss+=loss.item()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        class_logits_t.append(accelerator.gather(outputs['logits']).detach().cpu().numpy())

    class_logits = np.concatenate(class_logits_t)
    class_logits = class_logits[: len(train_dataset)]

    metrics, predicted_classes = compute_metrics_cls(
        None, None,class_logits, train_dataset, train_data
    )
    del class_logits,class_logits_t, outputs, batch
    torch.cuda.empty_cache()

    train_metrics = {'train_' + k:v for k,v in metrics.items()}

    # Evaluation
    model.eval()
    start_logits_t = []
    end_logits_t = []
    class_logits_t = []
    accelerator.print("Evaluation!")

    val_loss = 0
    val_qa_loss = 0
    val_cls_loss = 0
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)
            val_loss += outputs['loss'].item()

        class_logits_t.append(accelerator.gather(outputs['logits']).cpu().numpy())

    class_logits = np.concatenate(class_logits_t)
    class_logits = class_logits[: len(validation_dataset)]

    metrics, predicted_classes = compute_metrics_cls(
        None, None,class_logits, validation_dataset, val_data
    )

    val_metrics = {'val_' + k:v for k,v in metrics.items()}

    metrics = train_metrics
    for k,v in val_metrics.items():
      metrics[k] = v

    metrics['epoch'] = epoch

    train_loss = train_loss/len(train_dataloader)
    val_loss = val_loss/len(eval_dataloader)
    metrics['train_cls_loss'] = train_loss
    metrics['val_cls_loss'] = val_loss

    print("Epoch stats: ")
    print(metrics)

    df_metrics = pd.DataFrame(metrics.values()).T
    df_metrics.columns=metrics.keys()
    stats = pd.concat([stats,df_metrics])

    stats.to_csv("results_cls_only_bert/metrics.csv")

#    model.save_pretrained(f"./models_task2_/")

Starting epoch:  0


  0%|          | 0/404 [00:00<?, ?it/s]

  0%|          | 0/3199 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/3199 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

Epoch stats: 
{'train_class_accuracy': 0.5358, 'train_class_f1': 0.525, 'val_class_accuracy': 0.455, 'val_class_f1': 0.4093, 'epoch': 9, 'train_cls_loss': 0.9468631933231165, 'val_cls_loss': 1.078855068595321}


In [48]:
model.eval()
start_logits = []
end_logits = []
class_logits = []
accelerator.print("Evaluation!")

for batch in eval_dataloader:
    with torch.no_grad():
        outputs = model(**batch)
    class_logits.append(accelerator.gather(outputs['logits']).cpu().numpy())

class_logits = np.concatenate(class_logits)
class_logits = class_logits[: len(validation_dataset)]

metrics, predicted_classes = compute_metrics_cls(
    start_logits, end_logits,class_logits, validation_dataset, val_data)

Evaluation!


  0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:
metrics

In [49]:
model.eval()
start_logits = []
end_logits = []
class_logits = []
accelerator.print("Evaluation!")

for batch in tqdm(test_dataloader):
    with torch.no_grad():
        outputs = model(**batch)

    class_logits.append(accelerator.gather(outputs['logits']).cpu().numpy())

class_logits = np.concatenate(class_logits)

class_logits = class_logits[: len(test_dataset)]

predicted_classes = compute_metrics_cls(
    start_logits, end_logits, class_logits, test_dataset, test_data, predictOnly=True
)


Evaluation!


  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

In [50]:
df_test_submission = pd.DataFrame(columns=['id','spoilerType'])
df_test_submission['id'] = test_data['id']
df_test_submission['spoilerType'] = predicted_classes
df_test_submission['spoilerType'] = df_test_submission['spoilerType'].apply(lambda x: label_decoder[x])

df_test_submission.to_csv('results_cls_only_bert/submission_T1_new.csv',index=False)
df_test_submission.spoilerType.value_counts()

,count
spoilerType,
phrase,275
passage,79
multi,46
